## Simple MD without thermostat

In [1]:
from ase import Atoms
import numpy as np
from ase.calculators.general import Calculator
from ase.md.verlet import VelocityVerlet
from ase import units

### calculator

In [2]:
class OFDFT_calculator(Calculator):
    name = 'OFDFT_calculator'
    implemented_properties = ['forces']

    def __init__(self, forces):
        self.forces = forces
#         self.nAtoms = atoms.get_number_of_atoms()

    def get_forces(self, atoms=None):
        # call ofdft code via os subprocesses
        # input new coords, new density
        return(self.forces)

### MD run

In [3]:
# create atoms object

at1 = [0,0,0]
at2 = [0.5,0.5,0]
at3 = [0,0.5,0.5]
at4 = [0.5,0,0.5]
cell1 = [3.9691, 0, 0]
cell2 = [0, 3.9691, 0]
cell3 = [0, 0, 3.9691]

atoms = Atoms('Al4',
             positions=[at1,at2, at3, at4],
             cell=[cell1, cell2, cell3],
             pbc=True)

# is this correct or must I use something like this for crystals?

# # Set up a crystal
# atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
#                           symbol='Cu',
#                           size=(size, size, size),
#                           pbc=True)

forces_nuclei = np.array([[0.0,0,0],[0,0,0],[0,0,0],[0,0,0]])

In [4]:
# set the forces from the OFDFT code
# improvement: call ofdft in this function?
# get_forces reads forces from file
atoms.set_calculator(OFDFT_calculator(forces_nuclei))

In [5]:


# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, 5 * units.fs)  # 5 fs time step.

# run only for one step
f=dyn.run(1)

# get new positions for force calculations
atoms.get_positions()

array([[0. , 0. , 0. ],
       [0.5, 0.5, 0. ],
       [0. , 0.5, 0.5],
       [0.5, 0. , 0.5]])

In [6]:
# read atom positions from .ion file

file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/ion.restart'
content = []
with open(file, 'r') as f:
    content = f.readlines()
    
for i,l in enumerate(content):
    content[i] = l.strip('%')
    content[i] = content[i].strip('\n')
    
pos = []
for i,l in enumerate(content):
    if l != 'BLOCK POSITIONS_CART':
        pass
    else:
        for j in content[i+1:]:
            if j !=  'END BLOCK POSITIONS_CART':
                coords = j.split()[1:4]
                coords = [float(c) for c in coords]
                pos.append(coords)
            else:
                break
        break

In [7]:
pos

[[-1.02429756, -1.1033554, -1.00798242],
 [0.995740543, 1.26481621, -1.02773597],
 [-0.977291515, 0.863024832, 0.953715818],
 [1.00584869, -1.02448542, 1.0820023]]

files for ofdft:
- job.inpt (stays the same during CPMD)
- job.ion (structure: update position, lattice vectors?)
- density (update from CPMD)
- al.lda.recpot (pseudopotentials: stay the same during CPMD)

Which code to do APDFT calculations? (must support crystal structures)
Are local ofdft pseudopotentials compatible with other non-local pseudopotentials?

How to do MD runs in OFDFT for comparison?

How to run NVE, What is NVT coupled with Nose-Hoover?
Why are the positions negative, so close togethcer and different from initial structure?



- get forces, velocities from ofdft during python runtime
- propagate nuclei


            